# Making Folder for PNG's

In [ ]:
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            #Test Line
            os.makedirs(os.path.join(directory, 'translated_files'))
            print('Folder Created')
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
current_directory = os.getcwd() 
#Location of desired pdf's
where_pdf_located = ''
filename = ""

#Desired location of where the png's will be stored
desired_folder_location = os.path.join(current_directory, filename)

desired_folder_name = os.path.join(current_directory, where_pdf_located, filename)

createFolder(desired_folder_location)

# PDF to Image Splitter

In [ ]:
import fitz
def convert_pdf_to_image(fic):
    #open your file
    doc = fitz.open(fic)
    #iterate through the pages of the document and create a RGB image of the page
    for page in doc:
        pix = page.get_pixmap()
        pix.save(f"{filename}\%i.png" % page.number)
    print('PDF has been converted')
        
convert_pdf_to_image(filename+'.pdf')

## Preprocessing Image Steps

In [ ]:
#1.) make the image grayscaled
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def image_preprocessing(file_path):
    img = cv2.imread(file_path)
    gray_image = grayscale(img)
    #Actually making the image black and white
    thresh, img_bw = cv2.threshold(gray_image, 200, 230, cv2.THRESH_BINARY)
    return thresh, img_bw

#Dilation
def thick_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((1,1), np.uint8)
    #Thickening of text
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)

#Dilation
def thin_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((10,10), np.uint8)
    #Thinning of text
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)

## Actually translating the PDF's

In [ ]:
import os
from PIL import Image, ImageDraw, ImageFont
import PIL.Image
import textwrap
import cv2
import easyocr
from deep_translator import GoogleTranslator, single_detection
from wand.image import Image
from wand.drawing import Drawing
from wand.font import Font
from pdf2image import convert_from_path

#Location of desired font
font_location = ""

image_location = desired_folder_location

list_of_files = os.listdir(desired_folder_location)

for j in range(len(list_of_files)):
#for j in range(25, 30):
    if j%50==0 and j != 0:
        print(f"Page {j} is done")
    #Base Image
    mang_page = image_location+'\\'+str(j)+".png"
    
    #Processing the Image for OCR
    mang_page_processed = image_location+'\\'+str(j)+".png"
    
    img = cv2.imread(mang_page_processed)
    gray_image = grayscale(img)
    thresh, img_bw = cv2.threshold(gray_image, 200, 230, cv2.THRESH_BINARY)
    dilated_image = thick_font(img_bw)
    
    #Create the reader so that it can detect words in the image
    reader = easyocr.Reader(['ch_sim'], gpu = True, verbose=False)

    #For Text that goes right to left in a horizontal way
    in_line = True

    bounds = reader.readtext(dilated_image, paragraph=in_line)
    
    fontsize = 25
    textCharWidth = 500
    
    with Image(filename=mang_page) as original:
        with original.clone() as converted:
            with converted as canvas:
                for i in range(len(bounds)):
                    try:
                        textCharWidth = 500

                        captured_text = bounds[i][1]
                        translated = GoogleTranslator(source='chinese (simplified)', target='english').translate(captured_text)

                        #Finding the dimensions of the bounding box
                        width = abs(bounds[i][0][0][0] - bounds[i][0][2][0])
                        height = abs(bounds[i][0][0][1] - bounds[i][0][2][1])

                        font = ImageFont.truetype(font_location, fontsize)

                        text = textwrap.fill(translated, textCharWidth)
                        img_temp = PIL.Image.new("RGBA", (1,1))
                        draw = ImageDraw.Draw(img_temp)
                        textsize = draw.textsize(text, font)
                        
                        #Top-left point of the rectangle
                        left, top= bounds[i][0][0][0], bounds[i][0][0][1]
                        #if height>100 and len(translated)>60:
                        if len(translated) < 1:
                            continue
                        elif height>100:
                            while textsize[1]<height:
                                if textsize[0]<(0.8*width):
                                    fontsize+=2

                                textCharWidth-=3
                                font = ImageFont.truetype(font_location, fontsize)

                                translated_wrapped = textwrap.fill(translated, textCharWidth)

                                img_temp = PIL.Image.new("RGBA", (1,1))
                                draw = ImageDraw.Draw(img_temp)
                                textsize = draw.textsize(translated_wrapped, font)

                            text = textwrap.fill(translated, textCharWidth)

                            #Actually writing the text boxes
                            with Drawing() as context:
                                context.fill_color = 'white'
                                context.rectangle(left=left, top=top, width=width, height=height)
                                font = Font(font_location)
                                context(canvas)
                                canvas.caption(text, left=left, top=top, width=width, height=height, font=font, gravity='center')

                        elif len(translated)<30 or width>450:
                            with Drawing() as context:
                                context.fill_color = 'white'
                                context.rectangle(left=left, top=top, width=width, height=height)
                                font = Font(font_location)
                                context(canvas)
                                canvas.caption(translated, left=left, top=top, width=width, height=height, font=font, gravity='center')     
                        else:
                            while textsize[1]<height:
                                if textsize[0]<(0.8*width):
                                    fontsize+=2

                                textCharWidth-=3
                                font = ImageFont.truetype(font_location, fontsize)

                                translated_wrapped = textwrap.fill(translated, textCharWidth)

                                img_temp = PIL.Image.new("RGBA", (1,1))
                                draw = ImageDraw.Draw(img_temp)
                                textsize = draw.textsize(translated_wrapped, font)

                            text = textwrap.fill(translated, textCharWidth)

                            #Actually writing the text boxes
                            with Drawing() as context:
                                context.fill_color = 'white'
                                context.rectangle(left=left, top=top, width=width, height=height)
                                font = Font(font_location)
                                context(canvas)
                                canvas.caption(text, left=left, top=top, width=width, height=height, font=font, gravity='center')
                    except:
                        continue
                canvas.save(filename=desired_folder_location+'\\translated_files\\'+str(j)+'.pdf')
print('All pages are Translated')
                

## Merging files

In [ ]:
import os
source_dir = ""
desired_filename = filename

def merge_pdf(path_to_pdf, desired_filename):
    from PyPDF2 import PdfFileMerger

    merger = PdfFileMerger()
    source_dir = path_to_pdf
    list_of_files = os.listdir(source_dir)
    
    file_limit = 500000000
    current_file_size = 0

    for i in range(len(list_of_files)):
        current_file_size+=os.path.getsize(source_dir+'\\'+str(i)+'.pdf')
        if file_limit > current_file_size:
            merger.append(source_dir+'\\'+str(i)+'.pdf')
            
        else:
            current_file_size = os.path.getsize(source_dir+'\\'+str(i)+'.pdf')
            merger.write('Combined_translated'+'\\'+desired_filename+f' {i/len(list_of_files)}.pdf')
            print('First Set Done')

            merger = PdfFileMerger()
            merger.append(source_dir+'\\'+str(i)+'.pdf')

            

    merger.write('Combined_translated'+'\\'+desired_filename+f' {i/len(list_of_files)}.pdf')

    print('Pages Merged!')
    
merge_pdf(source_dir, desired_filename)